In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Drive functions

#load the track
df = pd.read_csv("track.csv")
TX = np.array(df["X"].to_list())
TY = np.array(df["Y"].to_list())

L = 5 #length of sensor array in cm
W = 10 #width of the car 
V = 5 #speed cm/s
B = 100 #baud rate 
VB = V/B #speed cm/cycle

#heading of car
def theta(vdiff, heading):
    heading += (2/W)*vdiff
    return heading

#velocity vector of car
def v(heading):
    return VB*np.cos(heading),  VB*np.sin(heading)

#coordinates of car
def p(heading, pos):
    x, y = v(heading)
    pos[0] = pos[0] + x
    pos[1] = pos[1] + y
    return pos[0], pos[1]

#nearest point on track 
def pl(x, y):
    distances = np.array([((TX[i] - x)**2 + (TY[i] - y)**2) for i in range(len(TX))])
    return TX[np.argmin(distances)], TY[np.argmin(distances)]

#rotate point 
def rotate(px, py, heading):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox = 0
    oy = 0
    qx = ox + np.cos(heading) * (px - ox) - np.sin(heading) * (py - oy)
    qy = oy + np.sin(heading) * (px - ox) + np.cos(heading) * (py - oy)
    return qx, qy

def findPos(pos, heading):
    phi = heading - 0.5*np.pi
    ax = pos[0] + 0.5*L*np.cos(phi)
    ay = pos[1] + 0.5*L*np.sin(phi)
    bx = pos[0] - 0.5*L*np.cos(phi)
    by = pos[1] - 0.5*L*np.sin(phi)

    #select points of interest (reduce time complexity)
    tx = TX[(min(ax, bx) < TX) & (TX < max(ax, bx))]
    ty = TY[(min(ay, by) < TY) & (TY < max(ay, by))]

    if len(tx) == 0 or len(ty) == 0:
        return pos

    #rotate the points
    ax, ay = rotate(ax, ay, phi)
    bx, by = rotate(bx, by, phi)
    for i in range(len(tx)):
        tx[i], ty[i] = rotate(tx[i], ty[i], phi)
    
    #test the rotation
    if not abs(bx - ax) < 0.1:
        exit()
    
    p1i = np.argmin(abs(tx - ax))
    p1x = tx[p1i]
    p1y = ty[p1i]
    tx[p1i] = tx[np.argmax(abs(tx - ax))]

    p2i = np.argmin(abs(tx - ax))
    p2x = tx[p2i]
    p2y = ty[p2i]

    #edge case (note bx - ax should be zero)
    if abs(p2x - p1x) == abs(bx - ax):
        return -100, -100
    
    #find point of intersection
    A = (p2y - p1y)/(p2x - p1x)
    B = p2y - A*p2x
    px = ax
    py = A*px + B

    #rotate back to original coordinate system
    px, py = rotate(px, py, -1*phi)

#calculate error
def err(pos, heading):

    cx = pos[0]
    cy = pos[1]

    px, py = findPos(pos, heading)

    ex = cx - px
    ey = cy - py
    e = (ex**2 + ey**2)**0.5
    if e > L:
        e = L
    if ex >= 0:
        return -1*e
    else:
        return e


In [3]:
tx = TX[:10]
ty = TY[:10]
findPos([0, 0], 1.6)

: 

In [ ]:
import time 

pos = [0,-12] #car starting position
diff = 0 #initial wheel speed differebce
heading = np.pi/2 #initial heading
pos_x = np.array([])
pos_y = np.array([])
errs = np.array([])

#PID constants
KP = VB/L
KI = 0
KD = 0

prev_e = 0
integral = 0
s = time.time()
while(pos[0] < 260 and time.time() - s < 20):
    #append the position history
    pos_x = np.append(pos_x, pos[0])
    pos_y = np.append(pos_y, pos[1])

    #find the car position
    heading = theta(diff, heading)
    pos[0], pos[1] = p(heading, pos)

    #find error
    e = err(pos, heading)
    errs = np.append(errs, e)

    #find wheel speed difference
    prop = KP*e
    int = KI*integral
    der = KD*(e - prev_e)
    diff = prop + int + der

    #update pid
    integral += e
    prev_e = e

plt.figure()
plt.plot(TX, TY)  
plt.plot(pos_x, pos_y)

plt.figure()
plt.plot(errs)

